In [1]:
# importing all the libraries
import os 
import zipfile
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.3)

In [3]:
path = 'C:\\Users\\LENOVO\\Desktop\\Hand Gesture\\images'

In [4]:
print(os.listdir(path))
data=[]
labels = []


['closedFist', 'fingerCircle', 'multiFingerBend', 'openPalm', 'semiOpenFist', 'semiOpenPalm', 'singleFingerBend']


In [5]:
# converting images to data points and saving it to array along with labels
for dir in os.listdir(path):
        for img_path in os.listdir(os.path.join(path,dir)):
             print(img_path)
             img = cv2.imread(os.path.join(path,dir,img_path))
             img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
             label = dir
             print(label)
             results = hands.process(img_rgb)
             print(results)
             if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    data_array=[]
                    for i in range(len(hand_landmarks.landmark)):
                        data_array.append(hand_landmarks.landmark[i].x) # appending the x coordinates
                        data_array.append(hand_landmarks.landmark[i].y) # appending the y coordinates
                data.append(data_array)
                labels.append(label)

IMG_20220430_180531.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_180538.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_180543.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_180553.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_180610.jpg
closedFist


c:\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_180617.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_180630.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_182912.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_182941.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_182944.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_182948.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_182950.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_182951.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220430_182952.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220501_151540.jpg
closedFist
<class 'mediapipe.python.solution_base.SolutionOutputs'>
IMG_20220501_151543.jpg

In [6]:
# saving the data to pickle file
import pickle
f = open('Hand_data.pickle','wb')
pickle.dump({'Data':data,'Label':labels},f)
f.close

<function BufferedWriter.close()>

In [ ]:
import pickle
data = pickle.load(open('/content/Hand_data.pickle','rb'))



In [ ]:
# importing the libraries to train the model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
# train_test split
x_train,x_test,y_train,y_test = train_test_split(data_array,labels_array,test_size=0.2,shuffle=True,stratify=labels_array)

In [ ]:
# using random forrest classifier
model = RandomForestClassifier()
model.fit(x_train,y_train)

In [ ]:
import joblib

path = '/content/HandGesture.joblib'

joblib.dump(model,path)

In [ ]:
y_predict = model.predict(x_test)
score = accuracy_score(y_test,y_predict)

In [ ]:
print(score)

0.7392857142857143

In [12]:
# testing in real time
import cv2
import mediapipe as mp 
import pickle
import numpy as np
import joblib

model = joblib.load('C:\\Users\\LENOVO\\Desktop\\Hand Gesture\\HandGesture (1).joblib')

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=False,min_detection_confidence=0.3)

cap = cv2.VideoCapture(0)
while True:
  ret , frame = cap.read()
  H,W,_ = frame.shape
 
  frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
  results = hands.process(frame_rgb)
            
  if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
      mp_drawing.draw_landmarks(
        frame,
        hand_landmarks,
        mp_hands.HAND_CONNECTIONS,
        mp_drawing_styles.get_default_hand_landmarks_style(),
        mp_drawing_styles.get_default_hand_connections_style())
    
    for hand_landmarks in results.multi_hand_landmarks:
                    data_array=[]
                    X=[]
                    Y=[]
                    for i in range(len(hand_landmarks.landmark)):
                        data_array.append(hand_landmarks.landmark[i].x) # appending the x coordinates
                        data_array.append(hand_landmarks.landmark[i].y) # appending the y coordinates
                        Y.append(hand_landmarks.landmark[i].y)
                        X.append(hand_landmarks.landmark[i].x)
    x1=int(min(X)*W)
    y1=int(min(Y)*H)

    x2 = int(max(X)*W)
    y2 = int(max(Y)*H)
    prediction = model.predict([np.asarray(data_array)])[0]
    prediction_text = str(prediction)
   


    
    cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,0),4)
    cv2.putText(frame,prediction_text,(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,1.3,(0,0,0),3,cv2.LINE_AA)
    cv2.imshow('frame',frame)
    if cv2.waitKey(25) & 0xFF == ord('w'):
      break

cap.release
cv2.destroyAllWindows


c:\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: Sy

<function destroyAllWindows>

: 